# Feels Protocol Simulation Calibration

This notebook calibrates the simulation parameters based on historical Solana CLMM data and generates baseline parameter configurations.

In [ ]:
import json
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Setup path handling for both local and online execution
try:
    # Try importing the package directly (works when installed via pip)
    from feels_sim.config import SimulationConfig
    from feels_sim.core import FeelsSimulation
    from feels_sim.plotting import setup_plot_style, DEFAULT_STYLE, create_figure_with_style, save_plot
    print("Package imported successfully")
    project_root = None  # Not needed when package is installed
    
except ImportError:
    # Fallback for local development - add project root to path
    print("Package not installed, using local development setup...")
    project_root = Path().cwd().parent
    sys.path.insert(0, str(project_root))
    
    try:
        from feels_sim.config import SimulationConfig
        from feels_sim.core import FeelsSimulation
        from feels_sim.plotting import setup_plot_style, DEFAULT_STYLE, create_figure_with_style, save_plot
        print("Local development imports successful")
    except ImportError as e:
        print(f"Import failed: {e}")
        print("Please ensure feels_sim package is installed or you're running from the project directory")
        raise

# Setup centralized plotting style
setup_plot_style()

print("Libraries imported with centralized plotting style configured")

## Historical Data Analysis (Placeholder)

In a production setup, this section would:
1. Pull historical swap data from Solana CLMMs (Orca Whirlpools, Raydium)
2. Analyze trade size distributions, frequency patterns, and liquidity utilization
3. Calibrate participant behavior parameters to match observed patterns

For now, we'll use reasonable baseline parameters based on DeFi market knowledge.

In [ ]:
# Placeholder for historical data analysis
# In production, this would pull from Solana RPC or data providers

def analyze_historical_trading_patterns():
    """Placeholder for historical data analysis."""
    # Mock data representing typical DeFi trading patterns
    return {
        'daily_volume_range': (50000, 500000),  # FeelsSOL
        'retail_trade_size_median': 100,
        'algo_trade_size_median': 2000,
        'trade_frequency_per_hour': 120,
        'lp_position_median': 10000,
        'fee_elasticity_coefficient': -1.5  # volume decreases as fees increase
    }

historical_patterns = analyze_historical_trading_patterns()
print("Historical trading patterns (mock data):")
for key, value in historical_patterns.items():
    print(f"  {key}: {value}")

## Baseline Parameter Configuration

Generate baseline parameters calibrated to typical DeFi market conditions.

In [ ]:
def create_baseline_config():
    """Create baseline simulation configuration."""
    
    # Market environment parameters (using new fee system)
    sim_config = {
        'initial_sol_price_usd': 100.0,
        'sol_volatility_daily': 0.05,  # 5% daily volatility
        'sol_trend_bias': 0.0,  # No directional bias
        'base_fee_bps': 30,  # 0.30% base fee
        'protocol_fee_rate_bps': 100,  # 1.0% protocol fee
        'creator_fee_rate_bps': 50,   # 0.5% creator fee
        'jitosol_yield_apy': 0.07,  # 7% APY
        'enable_participant_behavior': True
    }
    
    return {
        'simulation_config': sim_config,
        'calibration_metadata': {
            'created_date': '2024-10-16',
            'data_sources': ['mock_defi_patterns'],
            'version': '1.0.0',
            'description': 'Baseline configuration calibrated to typical DeFi market patterns'
        }
    }

baseline_params = create_baseline_config()
print("Baseline configuration created")
print(f"Protocol fee: {baseline_params['simulation_config']['protocol_fee_rate_bps']} bps")
print(f"Creator fee: {baseline_params['simulation_config']['creator_fee_rate_bps']} bps")
print(f"Base fee: {baseline_params['simulation_config']['base_fee_bps']} bps")

## Parameter Validation

Test the calibrated parameters to ensure they produce reasonable market behavior.

In [ ]:
def validate_parameters(config_dict):
    """Validate calibrated parameters by running short simulations."""
    
    # Create configuration object using new system
    sim_config = SimulationConfig(**config_dict['simulation_config'])
    
    # Run validation simulation
    print("Running validation simulation (48 hours)...")
    sim = FeelsSimulation(sim_config)
    results = sim.run(hours=48)
    
    # Calculate key metrics using new results structure
    total_volume = sum(s.volume_feelssol for s in results.snapshots)
    avg_hourly_volume = total_volume / 48
    final_floor = results.snapshots[-1].floor_price_usd
    final_state = results.snapshots[-1].floor_state
    
    validation_results = {
        'total_volume_48h': total_volume,
        'avg_hourly_volume': avg_hourly_volume,
        'final_floor_price': final_floor,
        'final_treasury_balance': final_state.treasury_balance,
        'final_buffer_balance': final_state.buffer_balance,
        'snapshots_count': len(results.snapshots)
    }
    
    print("Validation results:")
    for key, value in validation_results.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.2f}")
        else:
            print(f"  {key}: {value}")
    
    return validation_results

validation_results = validate_parameters(baseline_params)

# Check if results are reasonable
reasonable = (
    1000 < validation_results['total_volume_48h'] < 2000000 and  # Volume in reasonable range
    validation_results['final_floor_price'] > 0 and  # Floor price exists
    validation_results['final_treasury_balance'] > 0  # Treasury has accumulation
)

print(f"\nParameters appear reasonable: {reasonable}")

## Fee Elasticity Testing

Test fee elasticity to ensure participant behavior responds appropriately to fee changes.

In [ ]:
def test_fee_elasticity(base_config):
    """Test fee elasticity across different fee levels."""
    
    fee_levels = [10, 20, 30, 40, 50]  # basis points
    volumes = []
    
    print("Testing fee elasticity...")
    
    for fee_bps in fee_levels:
        # Create config for this fee level
        config = base_config['simulation_config'].copy()
        config['base_fee_bps'] = fee_bps
        
        sim_config = SimulationConfig(**config)
        
        # Run short simulation
        sim = FeelsSimulation(sim_config)
        results = sim.run(hours=24)
        
        total_volume = sum(s.volume_feelssol for s in results.snapshots)
        volumes.append(total_volume)
        
        print(f"  Fee {fee_bps}bps: Volume {total_volume:.0f}")
    
    # Calculate elasticity
    fee_changes = np.diff(fee_levels) / np.array(fee_levels[:-1])
    volume_changes = np.diff(volumes) / np.array(volumes[:-1])
    elasticity = volume_changes / fee_changes
    
    avg_elasticity = np.mean(elasticity)
    print(f"\nAverage fee elasticity: {avg_elasticity:.2f}")
    print(f"Volume decreases as fees increase: {volumes[-1] < volumes[0]}")
    
    return fee_levels, volumes, avg_elasticity

fee_levels, volumes, elasticity = test_fee_elasticity(baseline_params)

# Plot fee elasticity using centralized plotting system
fig, ax = create_figure_with_style(1, 1, DEFAULT_STYLE.figure_size_single)

# Get centralized color palette
colors = DEFAULT_STYLE.color_palette

# Main volume plot
ax.plot(fee_levels, volumes, 'o-', linewidth=DEFAULT_STYLE.linewidth * 2, markersize=10, 
        color=colors[0], markeredgecolor='white', markeredgewidth=DEFAULT_STYLE.marker_edgewidth, 
        alpha=DEFAULT_STYLE.alpha, label='Trading Volume')

# Add trend line
z = np.polyfit(fee_levels, volumes, 1)
p = np.poly1d(z)
ax.plot(fee_levels, p(fee_levels), "--", alpha=DEFAULT_STYLE.alpha - 0.1, 
        color=colors[1], linewidth=DEFAULT_STYLE.linewidth, 
        label=f'Trend (slope: {z[0]:.0f})')

ax.set_xlabel('Base Fee (basis points)', weight='bold')
ax.set_ylabel('24h Trading Volume (FeelsSOL)', weight='bold')
ax.set_title('Fee Elasticity: Volume vs Fee Level', weight='bold', pad=20)
ax.legend(frameon=True, fancybox=True, shadow=True)

# Add annotations using centralized styling
ax.annotate(f'Elasticity: {elasticity:.2f}', 
           xy=(0.02, 0.98), xycoords='axes fraction',
           bbox=dict(boxstyle='round,pad=0.5', facecolor='lightblue', alpha=0.8),
           fontsize=12, weight='bold', va='top')

plt.tight_layout()

# Save using centralized function
save_plot(fig, None, "fee_elasticity_calibration")
plt.show()

print(f"Fee elasticity validation: {'PASS' if elasticity < 0 else 'FAIL'} (should be negative)")

## Export Calibrated Parameters

Save the validated parameters to a configuration file for use in simulations.

In [ ]:
# Add validation results to metadata
baseline_params['calibration_metadata'].update({
    'validation_results': validation_results,
    'fee_elasticity': elasticity,
    'validation_passed': reasonable and elasticity < 0
})

# Setup export path for both local and online execution
if project_root is not None:
    # Local development
    output_path = project_root / 'experiments' / 'configs' / 'params_baseline.json'
else:
    # Online/installed package execution
    output_path = Path('experiments') / 'configs' / 'params_baseline.json'

output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(baseline_params, f, indent=2)

print(f"Baseline parameters exported to: {output_path}")
print(f"Configuration includes {len(baseline_params)} sections")
print(f"Validation passed: {baseline_params['calibration_metadata']['validation_passed']}")

## Summary

This calibration notebook:

1. Analyzes historical trading patterns (placeholder implementation)
2. Creates baseline participant behavior parameters
3. Validates parameters through simulation testing
4. Tests fee elasticity to ensure realistic behavior
5. Exports calibrated parameters for simulation use

**Next Steps:**
- Replace placeholder historical analysis with real Solana CLMM data
- Implement additional market scenarios (bull/bear/sideways)
- Add LP retention and arbitrage efficiency metrics
- Create parameter sensitivity analysis